What we need is a small parcel to work on, with a nice contourplot with critical points and minima found after initiating local method and then a 3d plot. 

In [1]:
using Pkg
Pkg.activate("../../.")
using Globtim
using DynamicPolynomials, DataFrames

  Activating project at `~/Globtim.jl`


In [8]:
# Constants and Parameters
const n, a, b = 2, 1, 6
const scale_factor = a / b   # Scaling factor appears in `main_computation`, maybe it should be a parameter.
const delta, alpha = 0.5, 1 / 10  # Sampling parameters
const tol_l2 = 3e-4            # Define the tolerance for the L2-norm
f = tref # Objective function

tref (generic function with 1 method)

One may assume that when we have access to exact evaluations, we would want to have a small $L^2$-norm tolerance `tol_l2 = 5e-4` and high probability of computing an accurate discrete $L^2$-norm `alpha= 1/10`.

We need to also return the number of samples used to generate the sample set. It is annoying that the error goes up while the degree has increased.

In [9]:
rand_center = [2*rand()-1, 2*rand()-1]*.75; # Random center

2-element Vector{Float64}:
 -0.024039250261503498
  0.4341011959605634

In [16]:
d = 14 # Initial Degree 
SMPL = 40 # Number of samples
TR = test_input(f, 
                dim = n,
                center=rand_center,
                GN=SMPL, 
                sample_range=scale_factor
                )
pol_cheb = Constructor(TR, d, GN=SMPL, basis=:chebyshev)
pol_lege = Constructor(TR, d, GN=SMPL, basis=:legendre);

@polyvar(x[1:n]); # Define polynomial ring 

current L2-norm: 0.007187621051931364
Number of samples: 40
current L2-norm: 0.007320701592476574
Number of samples: 40


Solve the system of partial derivatives using `Msolve`. 

In [17]:
msolve_polynomial_system(pol_cheb, x)
df_cheb = msolve_parser("outputs.ms", f, n, TR)

Input file already exists: inputs.ms
Output file already exists: outputs.ms



--------------- INPUT DATA ---------------
#variables                       2
#equations                       2
#invalid equations               0
field characteristic             0
homogeneous input?               0
signature-based computation      0
monomial order                 DRL
basis hash table resetting     OFF
linear algebra option            2
initial hash table size     131072 (2^17)
max pair selection             ALL
reduce gb                        1
#threads                         1
info level                       1
generate pbm files               0
------------------------------------------



---------------- TIMINGS ----------------
overall(elapsed)        0.00 sec
overall(cpu)            0.00 sec
select                  0.00 sec  13.2%
symbolic prep.          0.00 sec  24.7%
update                  0.00 sec   8.9%
convert                 0.00 sec  24.2%
linear algebra          0.00 sec  14.4%
reduce gb               0.00 sec   0.0%
-----------------------------------------

---------- COMPUTATIONAL DATA -----------
size of basis                    14
#terms in basis                2028
#pairs reduced                   14
#GM criterion                    91
#redundant elements               1
#rows reduced                    42
#zero reductions                  1
max. matrix data                 29 x 197 (75.144%)
max. symbolic hash table size  2^11
max. basis hash table size     2^16
-----------------------------------------


---------------- TIMINGS ----------------
overall(elapsed)        0.00 sec
overall(cpu)            0.00 sec
--------------------------------------

[0][1]Real root isolation starts at precision 204
Number of real roots: 139
Starts real root extraction.
<594><-<596><594><-<601><592>


------------------------------------------------------------------------------------------------------

---------- COMPUTATIONAL DATA -----------
#primes                         162
#bad primes                       0
-----------------------------------------

---------------- TIMINGS ----------------
CRT and ratrecon(elapsed)       0.00 sec
-----------------------------------------

---------------- TIMINGS ----------------
rational param(elapsed)         0.80 sec
rational param(cpu)             0.80 sec
-----------------------------------------

=== Starting MSolve Parser (dimension: 2) ===

=== Starting file processing (dimension: 2) ===
Raw content length: 39613
Cleaned content: [0,[1,[[[-138284470059330888739477389279488999887/2^123,1176337307062069680353545399331766302961/2^126],[-2832375840870408957506100858903383822812115136209405884518820116813/2^221,-708093960217602239376525214725845955703028784052351471129705029203/2^219]],[[-103914060925127104897783569012141437283/2^132,-4

<-<496><478>{97.84%}<478>Elapsed time (real root extraction) = 0.24


------------------------------------------------------------------------------------
msolve overall time           1.49 sec (elapsed) /  1.49 sec (cpu)
------------------------------------------------------------------------------------


Row,x1,x2,z
,Float64,Float64,Float64
1,0.0445915,0.294023,0.727706
2,-0.0272203,0.294079,-2.76344
3,0.0744906,0.294097,-1.52201
4,0.115916,0.294245,0.685146
5,-0.0777444,0.294252,0.158637
6,0.126816,0.294294,0.748306
7,0.139243,0.294353,0.791107
8,-0.126737,0.294489,-2.3143
9,-0.159948,0.294659,-1.32912


In [18]:
plot_polyapprox(pol_cheb, TR, df_cheb)

In [25]:
using GLMakie
function plot_polyapprox_rotate(pol::ApproxPoly, TR::test_input, df::DataFrame)
    # Extract coordinates and function values
    coords = pol.scale_factor * pol.grid .+ TR.center'
    z_coords = pol.z

    if size(coords)[2] == 2  # Plot if the dimensions are 2
        fig = Figure(size=(800, 600))
        ax = Axis3(fig[1, 1], title="Trefethen Function",
            xlabel="X-axis", ylabel="Y-axis", zlabel="Z-axis")

        # Create a regular grid for surface plotting
        x_unique = sort(unique(coords[:, 1]))
        y_unique = sort(unique(coords[:, 2]))

        # Initialize the Z grid with NaN
        Z = fill(NaN, (length(y_unique), length(x_unique)))

        # Fill the Z grid by matching coordinates
        for (idx, (x, y, z)) in enumerate(zip(coords[:, 1], coords[:, 2], z_coords))
            i = findlast(≈(y), y_unique)
            j = findlast(≈(x), x_unique)
            if !isnothing(i) && !isnothing(j)
                Z[j, i] = z
            end
        end

        # Create surface plot
        surface!(ax, x_unique, y_unique, Z,
            colormap=:viridis,
            transparency=true,
            alpha=0.8)

        # Plot the critical points
        scatter!(ax, df.x1, df.x2,
            df.z,
            markersize=10,
            color=:orange,
            label="Chebyshev approximant critical points")

        # Record rotating animation
        record(fig, "polyapprox_rotation.mp4", 1:240; framerate=30) do frame
            ax.azimuth[] = 1.7pi + 0.4 * sin(2pi * frame / 240)
            ax.elevation[] = pi / 4 + 0.3 * cos(2pi * frame / 240)
        end

        display(fig)
        return fig
    end
end

plot_polyapprox_rotate (generic function with 1 method)

In [26]:
plot_polyapprox_rotate(pol_cheb, TR, df_cheb)